In [11]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [12]:
import warnings
warnings.simplefilter("ignore")

In [13]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [14]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [15]:
mat_file_list=['arrhythmia.mat','cardio.mat','glass.mat','ionosphere.mat','letter.mat','lympho.mat','mnist.mat','musk.mat','optdigits.mat','pendigits.mat','pima.mat','satellite.mat','satimage-2.mat','shuttle.mat','vertebral.mat','vowels.mat','wbc.mat']

In [16]:
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

In [17]:
df_columns=['Data','#Sample','#Dimensions','Outlier Perc','PCA','MCD','OCSVM','LOF','CBLOF','KNN','HBOS','ABOD','IFOREST','FEATUREBAGGING']

In [18]:
df_columns

['Data',
 '#Sample',
 '#Dimensions',
 'Outlier Perc',
 'PCA',
 'MCD',
 'OCSVM',
 'LOF',
 'CBLOF',
 'KNN',
 'HBOS',
 'ABOD',
 'IFOREST',
 'FEATUREBAGGING']

In [19]:
roc_df=pd.DataFrame(columns=df_columns)
prn_df=pd.DataFrame(columns=df_columns)
time_df=pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: []


In [20]:
from time import time
random_state = np.random.RandomState(42)
for mat_file in mat_file_list:
    print("\n... Processing",mat_file,'.....')
    mat = loadmat(mat_file)
    
    X = mat['X']
    y = mat['y'].ravel()
    
    outliers_fraction = np.count_nonzero(y) / len(y)
    
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = random_state)
    
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
    
    classifiers = {'Angle-based Outlier Detector (ABOD)' : ABOD(contamination=outliers_fraction), 
                  'Cluster-based Local Outlier Factor': CBLOF(contamination=outliers_fraction,check_estimator=False ,random_state=random_state),
                  'Feature Bagging': FeatureBagging(contamination=outliers_fraction, random_state=random_state),
                  'Historgram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
                  'Isolation Forest' : IForest(contamination=outliers_fraction, random_state=random_state),
                  'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
                  'Local Outlier Factor (LOF)' : LOF(contamination=outliers_fraction),
                  'Minimum Covariance Determinant (MCD)': MCD(contamination=outliers_fraction, random_state=random_state),
                  'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
                  'Principal Component Analysis (PCA)': PCA(contamination=outliers_fraction, random_state=random_state)}
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1-t0, 4)
        time_list.append(duration)
        
        roc = round(roc_auc_score(y_test, test_scores), 4)
        prn = round(precision_n_scores(y_test, test_scores), 4)
        
        print(f"{clf_name} ROC: {roc}, precision @ rank n: {prn}, execution time: {duration}s' ")
        
        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat .....
Angle-based Outlier Detector (ABOD) ROC: 0.7687, precision @ rank n: 0.3571, execution time: 1.0701s' 
Cluster-based Local Outlier Factor ROC: 0.7684, precision @ rank n: 0.4643, execution time: 1.3827s' 
Feature Bagging ROC: 0.7799, precision @ rank n: 0.5, execution time: 0.4857s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.8511, precision @ rank n: 0.5714, execution time: 1.243s' 
Isolation Forest ROC: 0.8527, precision @ rank n: 0.5714, execution time: 0.3191s' 
K Nearest Neighbors (KNN) ROC: 0.782, precision @ rank n: 0.5, execution time: 0.0718s' 
Local Outlier Factor (LOF) ROC: 0.7787, precision @ rank n: 0.4643, execution time: 0.0608s' 
Minimum Covariance Determinant (MCD) ROC: 0.8228, precision @ rank n: 0.4286, execution time: 0.5425s' 
One-class SVM (OCSVM) ROC: 0.7986, precision @ rank n: 0.5, execution time: 0.0389s' 
Principal Component Analysis (PCA) ROC: 0.7997, precision @ rank n: 0.5, execution time: 0.0519s' 

... Processing

Isolation Forest ROC: 0.5764, precision @ rank n: 0.0161, execution time: 0.6542s' 
K Nearest Neighbors (KNN) ROC: 0.3824, precision @ rank n: 0.0, execution time: 1.4132s' 
Local Outlier Factor (LOF) ROC: 0.4584, precision @ rank n: 0.0484, execution time: 1.7394s' 
Minimum Covariance Determinant (MCD) ROC: 0.3486, precision @ rank n: 0.0, execution time: 1.0001s' 
One-class SVM (OCSVM) ROC: 0.4972, precision @ rank n: 0.0, execution time: 1.3873s' 
Principal Component Analysis (PCA) ROC: 0.504, precision @ rank n: 0.0, execution time: 0.0448s' 

... Processing pendigits.mat .....
Angle-based Outlier Detector (ABOD) ROC: 0.7008, precision @ rank n: 0.0308, execution time: 1.2557s' 
Cluster-based Local Outlier Factor ROC: 0.9609, precision @ rank n: 0.3077, execution time: 0.2084s' 
Feature Bagging ROC: 0.4687, precision @ rank n: 0.0462, execution time: 3.8208s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.9294, precision @ rank n: 0.2615, execution time: 0.0079s' 
Isolation Fores

In [21]:
roc_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8527,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9414,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8309,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5778,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,mnist,7603,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7801,0.8409,0.7085,0.863,0.8417,0.8396
0,musk,3062,166,3.1679,0.0809,1,0.5228,0.9999,0.9996,0.7348,0.5323,1,1,1
0,optdigits,5216,64,2.8758,0.4428,0.7852,0.4641,0.8822,0.5764,0.3824,0.4584,0.3486,0.4972,0.504
0,pendigits,6870,16,2.2707,0.7008,0.9609,0.4687,0.9294,0.9422,0.7602,0.481,0.8271,0.93,0.9332


In [22]:
time_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,1.0701,1.3827,0.4857,1.243,0.3191,0.0718,0.0608,0.5425,0.0389,0.0519
0,cardio,1831,21,9.6122,0.3082,0.1277,0.6223,0.006,0.3092,0.1277,0.0888,0.5545,0.0628,0.006
0,glass,214,9,4.2056,0.0549,0.0549,0.0409,0.002,0.2523,0.008,0.003,0.0299,0.002,0.001
0,ionosphere,351,33,35.8974,0.0549,0.0559,0.0918,0.009,0.2514,0.014,0.0061,0.0568,0.004,0.0039
0,letter,1600,32,6.25,0.2813,0.0957,0.6004,0.008,0.3082,0.1047,0.0718,0.9136,0.0638,0.005
0,lympho,148,18,4.0541,0.0209,0.0329,0.0239,0.004,0.2424,0.006,0.002,0.0259,0.001,0.002
0,mnist,7603,100,9.2069,5.6509,0.8108,40.1336,0.0389,1.4551,5.212,5.2142,2.2042,3.7081,0.1147
0,musk,3062,166,3.1679,1.7912,0.2932,11.4903,0.0668,1.1858,1.4651,1.7383,11.4414,1.134,0.1496
0,optdigits,5216,64,2.8758,2.3827,0.4009,12.2383,0.0249,0.6542,1.4132,1.7394,1.0001,1.3873,0.0448
0,pendigits,6870,16,2.2707,1.2557,0.2084,3.8208,0.0079,0.5097,0.4987,0.4538,1.7413,0.7131,0.0065


In [23]:
prn_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5714,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.5,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,mnist,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.2979,0.4144,0.339,0.3973,0.3801,0.3767
0,musk,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9333,0.2333,0.1333,0.9667,1,1
0,optdigits,5216,64,2.8758,0.0161,0,0.0484,0.2581,0.0161,0,0.0484,0,0,0
0,pendigits,6870,16,2.2707,0.0308,0.3077,0.0462,0.2615,0.2769,0.0462,0.0462,0.0615,0.2923,0.3385
